### Signing Process

A signing process example for a message-bit b = 0 using Gottesman-Chuang scheme
Let Person A (Alice) want to send a message to Person B (Bob). Hash algorithms won't be considered, so Alice has to sign every single bit of her message. Message-Bit b 
∈
{
0
,
1
}
\in \{0,1\}.

Alice chooses M pairs of private keys 
{
k
0
i
,
k
1
i
}
1
≤
i
≤
M
\{k_{0}^{i},k_{1}^{i}\}\quad 1\leq i\leq M

All the 
k
0
k_{0} keys will be used to sign the message-bit if b = 0.
All the 
k
1
k_{1} keys will be used to sign the message-bit if b = 1.
The function which maps 
k
↦
|
f
k
⟩
k\mapsto |f_{k}\rangle  is known to all parties. Alice now computes the corresponding public keys 
{
|
f
k
0
i
⟩
,
|
f
k
1
i
⟩
}
\{|f_{{k_{0}}}^{i}\rangle ,|f_{{k_{1}}}^{i}\rangle \} and gives all of them to the recipients. She can make as many copies as she needs, but has to take care, not to endanger the security 
(
n
≫
T
m
 has to hold 
)
\left(n\gg Tm{\text{ has to hold }}\right).

Her level of security limits the number of identical public keys she can create

If message-bit b = 0, she sends all her private keys 
k
0
k_{0} along with the message-bit b to Bob
message-bit b = 1, she sends all her private keys 
k
1
k_{1} along with the message-bit b to Bob
Remember: In this example Alice picks only one bit b and signs it. She has to do that for every single bit in her message

In [177]:
from qiskit import *
from qiskit_ionq_provider import IonQProvider 
from qiskit.providers.jobstatus import JobStatus
#Call provider and set token value
provider = IonQProvider(token='BFmvdArkiCbsS12r4LZf5VgYDo4HngsS')
from random import randrange

# from swap_test import swaptest as swap_test

%run swap_test.ipynb
%run Hash_Function.ipynb

In [178]:
# given a message, produces a Quantum Digital Signature
class Signed_Transaction():
    def __init__(self, message):
        self.message = message
        self.bitstring = self.string_to_bitstring(message)
        self.signed_transaction = self.sign_transaction()
        
    def string_to_bitstring(self, m):
        bs = ""
        for i in bytes(m, encoding='utf-8'):
            bs += bin(i)[2:]
        return bs
    
        
    # make M pairs of private keys
    def generate_priv_keys(self):
        n = 4
        d = 3
        
        return {
            'k0': [bitstring(randrange(2**n - 1),n) for i in range(d)],
            'k1': [bitstring(randrange(2**n - 1),n) for i in range(d)]
        }
    
    # make M pairs of public keys (Quantum States)
    def generate_pub_keys(self, priv_keys):
        return {
            "f_k0":[crypto_hash(ki, priv_keys["k0"]) for ki in priv_keys["k0"]], 
            "f_k1":[crypto_hash(kj, priv_keys["k1"]) for kj in priv_keys["k1"]]
        }
    
    def sign_bit(self, b, priv_keys, pub_keys):
        
        return {
            "m_bit" : b,
            "priv_keys" : priv_keys['k0'] if b == '0' else priv_keys['k1'],
            "pub_keys" : pub_keys
        }
    
    def sign_transaction(self):
        signed_transaction = []
        
        for b in self.bitstring:
            priv_keys = self.generate_priv_keys()
            pub_keys = self.generate_pub_keys(priv_keys)
            
            signed_transaction.append(self.sign_bit(b, priv_keys, pub_keys))
            
        return signed_transaction
        
    def get_signature(self):
        return self.signed_transaction

In [189]:
class Validation():
    
    BIT_THRESHOLD = .3 #???
    MESSAGE_THRESHOLD = .1 #??????
    
    def __init__(self, transaction):
        self.signature = transaction.get_signature()
        #self.validations = self.validate_transaction()

    def key_tests(self, signed_bit):
        priv_keys = signed_bit["priv_keys"]
        
        # select list of public keys corresponding to bit value
        pub_keys = signed_bit["pub_keys"]['f_k0'] if signed_bit["m_bit"] == '0' else signed_bit["pub_keys"]['f_k1']
        
        # hash private keys
        hashed_keys = [crypto_hash(ki, priv_keys) for ki in priv_keys]
        
        # perform swap test on public and hashed private keys
        tests = [swap_test(pki, pkj) for pki, pkj in zip(pub_keys, hashed_keys)]
        
        return tests
    
    def validate_bit(self, signed_bit):
        
        tests = self.key_tests(signed_bit)
        #print(tests)
        
        M = len(tests)
        threshold  = M * self.BIT_THRESHOLD
        
        r = sum(tests)
        if r < threshold:
            passed = True
        else:
            passed = False
            
            
        return int(passed)
    
#         return {
#             "passed": passed,
#             "r": r,
#             "threshold": threshold,
#             "tests": tests,
#         }
    
    def validate(self):
        validations = []
        
        for signed_bit in self.signature:
            validations.append(self.validate_bit(signed_bit))
                    
        failures = len(validations) - sum(validations)
        failure_rate = failures / len(validations)
        
        return {
            "pass": failure_rate < self.MESSAGE_THRESHOLD,
            "bits_failure": str(round(failure_rate * 100, 4)) + '%'
        }
            

In [190]:
class Forgery(Signed_Transaction):
    def __init__(self, message):
        self.message = message
        self.bitstring = self.string_to_bitstring(message)
        self.forged_transaction = self.forge_transaction()
        
    def forge_transaction(self):
        signed_transaction = []
        
        for b in self.bitstring:
            # generate a set of public keys and forget corresponding private
            pub_keys = self.generate_pub_keys(self.generate_priv_keys())
            # forge a new set of private keys
            priv_keys = self.generate_priv_keys()
            
            signed_transaction.append(self.sign_bit(b, priv_keys, pub_keys))
            
        return signed_transaction
        
    def get_signature(self):
        return self.forged_transaction

In [191]:
signed_message = Signed_Transaction("1234556")
forged_message = Forgery("1234556")

In [192]:
good_validation = Validation(signed_message)
print(good_validation.validate())

bad_validation = Validation(forged_message)
print(bad_validation.validate())

{'pass': True, 'bits_failure': '0.0%'}
{'pass': False, 'bits_failure': '69.0476%'}


In [125]:
# passes = 0
# fails = 0

# for i in range(5):
#     # Message bit
#     b = 0

#     sig = Signature(b)
#     sig.get_signature()['pub_keys']['f_k1']
#     val = Validation(sig)
    
    
#     result = val.result()

#     if result['passed']:
#         passes += 1
#     else:
#         fails += 1

# print(passes, fails)

In [194]:
pubk = signed_message.get_signature()[4]['pub_keys']['f_k1'][1]

from qiskit_textbook.tools import array_to_latex
array_to_latex(pubk, pretext="\\text{Statevector} = ")

<IPython.core.display.Math object>

In [127]:
signed_message.get_signature()[8]['pub_keys']['f_k0']

[array([-0.69351992+0.j,  0.69351992+0.j, -0.13794969+0.j, -0.13794969+0.j]),
 array([ 0.5-3.06161700e-16j,  0.5+3.06161700e-16j, -0.5-1.01051188e-31j,
         0.5-1.13514263e-31j]),
 array([ 0.5879378 +2.48714532e-17j, -0.5879378 +3.35463213e-16j,
         0.39284748-9.25662020e-16j,  0.39284748-9.97337041e-16j])]

In [157]:
2**7

128